In [1]:
import pandas as pd
texts = pd.read_csv("textDiscourse.csv")

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    # newString = re.sub(r"'s\b","",newString)
    # # remove punctuations
    # newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

In [2]:
textDiscourse = []
totalTexts = len(texts)
counter = 1

for text in texts:
     #cursor.execute(f'SELECT explicit_spelling FROM text_discourse where text_uuid = "{text[1]}" and explicit_spelling is not null and explicit_spelling not like "%…%" and explicit_spelling not like "%x%" and type = "word" order by word_on_tablet')
    discourse = text_cleaner(' '.join(texts['words']))
    if len(discourse.split(' ')) > 50:  
        textDiscourse.append({'uuid': text[1], 'words': discourse})
    #print(f'{counter/totalTexts*100}% complete')
    #counter = counter + 1

In [3]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical, pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [4]:
import re



def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    # print('Total Sequences: %d' % len(sequences))
    return sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [5]:
# preprocess the text
data_new = ''
model = Sequential()
totalTexts = len(textDiscourse)
counter = 1

for text in textDiscourse:
    data_new = text_cleaner(text['words'])
    # create sequences   
    sequences = create_seq(data_new)
    # create a character mapping index
    chars = sorted(list(set(data_new)))
    mapping = dict((c, i) for i, c in enumerate(chars))
    tempSequences = list()
    for line in sequences:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        tempSequences.append(encoded_seq)
    sequences = tempSequences
    # vocabulary size
    vocab = len(mapping)
    sequences = np.array(sequences)
    # create X and y
    X, y = sequences[:,:-1], sequences[:,-1]
    # one hot encode y
    y = to_categorical(y, num_classes=vocab)
    # create train and validation sets
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

    # print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)
    model.add(Embedding(vocab, 50, input_length=30, trainable=True))
    # model.add(LSTM(100, return_sequences=True))
    # model.add(LSTM(100))    
    model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
    model.add(Dense(vocab, activation='softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
    # fit the model
    model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))
    print(f'{counter/totalTexts*100}% complete')
    counter = counter + 1
    

Epoch 1/100


KeyboardInterrupt: 